In [ ]:
# When the user query is for a web search
# ReAct loop implementation with few-shot prompting
def react_agent(query, max_steps=5):
    """
    ReACT agent that uses Serper API for web search

    """

# Few-shot examples for ReAct pattern
    few_shot = """

    Example:
    User: What is the ingredient list of the Olay Super Cream With Sunscreen SPF 30?
    Thought: I should look up the ingredient list of the Olay Super Cream With Sunscreen SPF 30.
    Action: Search("ingredient list of the Olay Super Cream With Sunscreen SPF 30")
    Observation: [Search results for 'ingredient list of the Olay Super Cream With Sunscreen SPF 30': \
    'There are a total of 4 ingredients, as listed on INCIdecoder::

    1:Water
    2:Zea Mays (Corn) Starch
    3:Glycerin
    4:Niacinamide*
    ']
    Thought: The search result gives the ingredient list clearly.
    Final Answer: There are a total of 4 ingredients, as listed on INCIdecoder::

    1:Water
    2:Zea Mays (Corn) Starch
    3:Glycerin

    User: What is the ingredient list of The Body Shop Camomile Sumptuous Cleansing Butter?
    Thought: I should look up the ingredient list of The Body Shop Camomile Sumptuous Cleansing Butter.
    Action: Search("ingredient list of The Body Shop Camomile Sumptuous Cleansing Butter")
    Observation: [Search results for 'ingredient list of The Body Shop Camomile Sumptuous Cleansing Butter': \
    'There are a total of 3 ingredients, as listed on INCIdecoder::

    1:Ethylhexyl Palmitate (Skin Conditioning Agent)
    2:Synthetic Wax (Binder/​Emollient)
    3:PEG-20 Glyceryl Triisostearate (Skin Conditioning Agent)
    ']
    Thought: The search result gives the ingredient list clearly.
    Final Answer: There are a total of 3 ingredients, as listed on INCIdecoder::

    1:Ethylhexyl Palmitate (Skin Conditioning Agent)
    2:Synthetic Wax (Binder/​Emollient)
    3:PEG-20 Glyceryl Triisostearate (Skin Conditioning Agent)

    """
    # Initialize the context with few-shot examples + the user query
    context = "These are few examples of sample answers:\n" + few_shot + "\nThis is a user query. Stop after generating Thought: and Action: \n" + f"User: {query}\nThought:"

    for step in range(max_steps):
        text = generate_response(context)
        print(f"text: ", text)
        # For some reason, the model is not adding context to its output. Hence, no need to extract from len(context) till end. from in-class code.

        # Parse the model's action or final answer
        action_match = re.search(r"Action: (.+)", text)
        final_match = re.search(r"Final Answer: ((.|\n)*)", text)


        if action_match:
            # print("Action match found")
            action = action_match.group(1).strip()
            if action.startswith("Search("):
                # Extract query term and invoke search tool
                query_term = action[len("Search("):-1].strip("'\"")
                print(f"Searching for: {query_term}")
                # observation = await get_ingredient_list(query_term)
                observation = asyncio.run(get_ingredient_list(query_term))
                print(f"Search results: {observation}")
                # Append Observation and continue
                print(f"Second text: {text}")
                context += text + f"\nObservation: {observation}\nThought:"
                continue

        if final_match:
            return final_match.group(1).strip()

            # If no actionable directive, break
            break

    return "No answer found."